pipeline的工作流程
1. 获得scheduler实例
2. 获得模型
3. 生成噪声
4. 迭代降噪
5. 显示图片

In [ ]:
!pip install diffusers accelerate transformers

获得scheduler实例

In [ ]:
from diffusers import DDPMScheduler, UNet2DModel

scheduler = DDPMScheduler.from_pretrained("google/ddpm-cat-256")

scheduler

获得model实例,并使用cuda加速

In [ ]:
model = UNet2DModel.from_pretrained("google/ddpm-cat-256", use_safetensors=True).to("cuda")

生成噪声

In [ ]:
import torch

noise = torch.randn(1, 3, model.config.sample_size, model.config.sample_size).to("cuda")

迭代去噪声

In [ ]:
scheduler.set_timesteps(num_inference_steps=50)

input = noise
for t in scheduler.timesteps:
  with torch.no_grad():
    noisy_residual = model(input, timestep=t).sample
  previous_noisy_sample = scheduler.step(model_output=noisy_residual, timestep=t, sample=input).prev_sample
  input = previous_noisy_sample

显示获取的图像

In [ ]:
from PIL import Image
import numpy as np

image = (input / 2 + 0.5).clamp(0, 1).squeeze()
image = (image.permute(1, 2, 0) * 255).round().to(torch.uint8).cpu().numpy()
image = Image.fromarray(image)

image